### Vector stores and retrievers

This video tutorial will familiarize you with langchain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data-- from (vector) databases and other sources -- for intergration with llm workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.

we will cover

 - Documents 
 - vector stores 
 - retriever 
 

In [ ]:
from lanf